In [1]:
!pip install pymongo
!pip install tensorflow==2.16.1
!pip install keras==3.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found

In [2]:
from pymongo import MongoClient
from tqdm import tqdm
import json
import pandas as pd
import numpy as np

In [3]:
import json
with open('my_dict.json', 'r') as file:
    data = json.load(file)

In [4]:
client = MongoClient('mongodb://49.13.173.177:27020/')
embs = client.sponsoredbye.embeddings

embeddings = []
labels = []

for n in tqdm(data['15']):
  temp_embs = []
  cursor = embs.find({'videoID':n})
  if cursor[0]['label'] == False:
    continue
  l = cursor[0]['label']
  e =  cursor[0]['embeddings']
  for m in e:
    temp_embs.append(m['embedding'])
  embeddings.append(temp_embs)
  labels.append(l)


100%|██████████| 1978/1978 [13:37<00:00,  2.42it/s]


# Neural network classifier

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Masking,TimeDistributed, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Extract embeddings and labels + padding

In [6]:
max_len = 1682
vector_dim = 768

X_padded = pad_sequences(embeddings,maxlen=max_len,dtype='float32', padding='post')

In [56]:
from keras.models import load_model
model = load_model('.keras')

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ masking_1 (Masking)                  │ (None, 1682, 768)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 1682, 100)           │         327,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 1682, 100)           │          60,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1682, 100)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 1682, 1)             │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,164,305 (4.44 MB)

 Trainable params: 388,101 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 776,204 (2.96 MB)

### neural network structure

In [57]:
predictions = model.predict(X_padded)
binary_predictions = (predictions >= 0.5).astype(int)

62/62 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step


In [58]:
# remove the padded part from the results. Note that I used directly the unpadded labels  to get the lenghts
cr_bin_predictions = []
for j,n in enumerate(labels):
    i = len(n)
    cr_bin_predictions.append(binary_predictions[j][:i])


In [59]:
from sklearn.metrics import f1_score
predictions_ = [x for vec in cr_bin_predictions for x in vec]
real_ = [x for vec in labels for x in vec]
print(len(predictions_),len(real_))
f1_score(real_, predictions_)

436742 436742


0.9212118108480989

In [40]:
results = {'real': labels,
           'prediction':[ x.tolist() for x in cr_bin_predictions]
}

with open('results_5.json', 'w') as file:
    json.dump(results, file, indent=4)
